<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/experimental/Elder_Impulse_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install pandas-ta==0.2.45b

     |████████████████████████████████| 6.3 MB 8.3 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.64-py2.py3-none-any.whl size=24109 sha256=b515e2ce11dfba1e305da1a7f204f3332d20cafa948811c5428e4f2ec54f6358
  Stored in directory: /root/.cache/pip/wheels/86/fe/9b/a4d3d78796b699e37065e5b6c27b75cff448ddb8b24943c288
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
     |████████████████████████████████| 88 kB 4.0 MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.2.45b0-py3-none-any.whl size=175303 sha256=b9b2827f4974681f138cb0924128407b919b754ab3998a0907edf0242e7d811c
  Stored in directory: /root/.cache/pip/wheels/b1/49/60/e674e28ae57160651c6198b76e1714a0b175e44e022fcd5137
Successfully built pandas-ta


In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_ta as ta
import warnings 
warnings.filterwarnings("ignore")

In [3]:
START = "2020-01-01"  #yyyy-mm-dd
#END = '2020-05-01'

In [4]:
tker = '^nsei' #'^nsei'
nifty =yf.download(tker,start=START)
nifty=nifty.dropna()
nifty=nifty.iloc[:,:4]


[*********************100%***********************]  1 of 1 completed


ElderImpulse
Impulse System for the price bars

Bullish price bar: (13-period EMA > previous 13-period EMA) and (MACD-Histogram > previous period's MACD-Histogram)
Bearish price bar: (13-period EMA < previous 13-period EMA) and (MACD-Histogram < previous period's MACD-Histogram)
Neutral price bar: when conditions for a bullish price bar or bearish price bar are not met.

In [5]:
nifty.ta.macd(append=True)
nifty.ta.ema(length=13,append=True)
nifty.ta.ema(length=50,append=True)
print()

In [6]:
nifty['Impulse']=0

In [7]:
for i in range(len(nifty)):
  if (nifty['EMA_13'].iloc[i]>nifty['EMA_13'].iloc[i-1] and nifty['MACDh_12_26_9'].iloc[i]>nifty['MACDh_12_26_9'].iloc[i-1]):
    nifty['Impulse'].iloc[i]=1
  elif (nifty['EMA_13'].iloc[i]<nifty['EMA_13'].iloc[i-1] and nifty['MACDh_12_26_9'].iloc[i]<nifty['MACDh_12_26_9'].iloc[i-1]):
    nifty['Impulse'].iloc[i]=-1
  else:
    nifty['Impulse'].iloc[i]=0    

In [8]:
nifty['Impulse'].value_counts()

 0    202
 1    168
-1     83
Name: Impulse, dtype: int64

In [9]:
nifty[-1:]

,open,high,low,close,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,EMA_13,EMA_50,Impulse
date,,,,,,,,,,
2021-10-28,18187.650391,18190.699219,17799.449219,17857.25,178.643978,-51.652845,230.296823,18095.312358,17542.427446,-1


In [10]:
nifty['signal50']=0

In [12]:
for i in range(len(nifty)):
  if (nifty['close'].iloc[i]>nifty['EMA_50'].iloc[i]):
    nifty['signal50'].iloc[i]=1
  elif (nifty['close'].iloc[i]<nifty['EMA_50'].iloc[i]):
    nifty['signal50'].iloc[i]=-1
  else:
    nifty['signal50'].iloc[i]=0 

In [13]:
nifty.tail(2)

,open,high,low,close,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,EMA_13,EMA_50,Impulse,signal50
date,,,,,,,,,,,
2021-10-27,18295.849609,18342.050781,18167.900391,18210.949219,218.016511,-25.193523,243.210034,18134.989417,17529.577546,0,1
2021-10-28,18187.650391,18190.699219,17799.449219,17857.250000,178.643978,-51.652845,230.296823,18095.312358,17542.427446,-1,1


In [14]:
nifty[['Impulse','signal50']].to_csv('impulse.csv')